# HOMEWORK 1


In [68]:
import pandas as pd
import duckdb as ddb
import yfinance as yf
import numpy as np
import re


## Question 1: [Index] S&P 500 Stocks Added to the Index
Which year had the highest number of additions?

In [69]:
# Pull the table from WIKI as dataframe
wiki_tables = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
sp500_df = wiki_tables[0]              
sp500_df

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
498,XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components,"White Plains, New York",2011-11-01,1524472,2011
499,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
500,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
501,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927


In [70]:
# Extract year from "Date Added" column
sp500_df['year_added'] = pd.to_datetime(sp500_df['Date added']).dt.year

# Find out value counts for each year added
sp500_df.year_added.value_counts()

year_added
1957    53
2016    23
2017    23
2019    22
2008    17
2022    16
2024    16
2023    15
2021    15
2012    14
1997    14
2015    14
2018    14
2002    13
2007    12
2009    12
2020    12
1998    11
1976    11
2013    10
2006    10
2011    10
2000     9
1999     9
2010     9
2001     8
2014     8
1994     7
2005     7
1995     7
1985     7
2004     6
1982     5
1989     5
2025     5
1984     5
2003     5
1988     4
1986     3
1993     3
1983     3
1981     3
1992     3
1980     3
1972     2
1987     2
1969     2
1996     2
1965     2
1973     2
1975     2
1970     2
1979     2
1974     1
1991     1
1964     1
1978     1
Name: count, dtype: int64

## Question 2. [Macro] Indexes YTD (as of 1 May 2025)
How many indexes (out of 10) have better year-to-date returns than the US (S&P 500) as of May 1, 2025?

In [71]:

tickers = {
    "S&P 500":         "^GSPC",
    "Shanghai (CN)":   "000001.SS",
    "Hang Seng (HK)":  "^HSI",
    "ASX 200 (AU)":    "^AXJO",
    "Nifty 50 (IN)":   "^NSEI",
    "TSX (CA)":        "^GSPTSE",
    "DAX (DE)":        "^GDAXI",
    "FTSE 100 (UK)":   "^FTSE",
    "Nikkei 225 (JP)": "^N225",
    "IPC Mexico (MX)": "^MXX",
    "Ibovespa (BR)":   "^BVSP"
}

start_date = "2025-01-01"
end_date   = "2025-05-01"



In [72]:
# For each ticker, download just that one; find first & last valid Close
ytd_returns = {}

for name, ticker in tickers.items():
    # Download one ticker
    data = yf.download(ticker, start=start_date, end=end_date, progress=False)["Close"]

    # Find the first non-NaN index (first trading day)
    first_idx = data.first_valid_index()
    # Find the last non-NaN index (last trading day)
    last_idx  = data.last_valid_index()

    if (first_idx is not None) and (last_idx is not None):
        # Pull out the float values directly:
        first_price = float(data.loc[first_idx])
        last_price  = float(data.loc[last_idx])
        ytd = (last_price - first_price) / first_price * 100.0
    else:
        # No valid prices at all mark as NaN
        ytd = float("nan")

    ytd_returns[name] = ytd



/var/folders/ps/lmbtyvg537g3811drk5y9p340000gn/T/ipykernel_73336/1868484086.py:15: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  first_price = float(data.loc[first_idx])
/var/folders/ps/lmbtyvg537g3811drk5y9p340000gn/T/ipykernel_73336/1868484086.py:16: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  last_price  = float(data.loc[last_idx])
/var/folders/ps/lmbtyvg537g3811drk5y9p340000gn/T/ipykernel_73336/1868484086.py:15: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  first_price = float(data.loc[first_idx])
/var/folders/ps/lmbtyvg537g3811drk5y9p340000gn/T/ipykernel_73336/1868484086.py:16: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future.

In [73]:
ytd_returns

{'S&P 500': -5.10330074824504,
 'Shanghai (CN)': 0.5048172440500021,
 'Hang Seng (HK)': 12.720017835921466,
 'ASX 200 (AU)': -0.9145002952478493,
 'Nifty 50 (IN)': 2.4904237409784913,
 'TSX (CA)': -0.22612571792915093,
 'DAX (DE)': 12.34637838149953,
 'FTSE 100 (UK)': 2.8425901908435485,
 'Nikkei 225 (JP)': -8.297930804200304,
 'IPC Mexico (MX)': 13.049444457570319,
 'Ibovespa (BR)': 12.438709677419354}

In [74]:
# Count how many of the OTHER ten have ytd > spx_return
beat_count = 0
spx_return = ytd_returns["S&P 500"]
for name, ret in ytd_returns.items():
    if name == "S&P 500":
        continue
    # comparison
    if (not pd.isna(ret)) and (ret > spx_return):
        beat_count += 1

beat_count

9

In [75]:
# Tabulate all results
df = (
    pd.DataFrame.from_dict(ytd_returns, orient="index", columns=["YTD (%)"])
      .sort_values("YTD (%)", ascending=False)
)

print(f"S&P 500 YTD (Jan→May 2025): {spx_return:.2f}%\n")
print("All indices’ YTD returns (sorted):")
print(df.to_string(float_format=lambda x: f"{x:>7.2f}%"))
print(f"\nNumber of indexes (out of 10) beating S&P 500: {beat_count}")

S&P 500 YTD (Jan→May 2025): -5.10%

All indices’ YTD returns (sorted):
                 YTD (%)
IPC Mexico (MX)   13.05%
Hang Seng (HK)    12.72%
Ibovespa (BR)     12.44%
DAX (DE)          12.35%
FTSE 100 (UK)      2.84%
Nifty 50 (IN)      2.49%
Shanghai (CN)      0.50%
TSX (CA)          -0.23%
ASX 200 (AU)      -0.91%
S&P 500           -5.10%
Nikkei 225 (JP)   -8.30%

Number of indexes (out of 10) beating S&P 500: 9


## Question 3. [Index] S&P 500 Market Corrections Analysis
Calculate the median duration (in days) of significant market corrections in the S&P 500 index.

In [76]:
# Download
raw = yf.download("^GSPC", start="1950-01-01", end="2025-05-01", progress=False)

# Handling close 
close_raw = raw[["Close"]]   
close_series = close_raw.squeeze()  
df = close_series.to_frame(name="close")
df.index.name = "date"
df = df.sort_index()

# Compute running max, flag ATH
df["running_max"] = df["close"].cummax()
df["is_ath"]     = df["close"] == df["running_max"]
ath_dates        = df.index[df["is_ath"]].tolist()

corrections = []
for i in range(len(ath_dates) - 1):
    start_ath = ath_dates[i]
    next_ath  = ath_dates[i + 1]
    window = df.loc[start_ath:next_ath, "close"]
    window = window[window.index > start_ath]
    window = window[window.index < next_ath]
    if window.empty:
        continue
    trough_price = float(window.min())
    trough_date  = window.idxmin()
    ath_price    = float(df.at[start_ath, "close"])
    drawdown_pct = (ath_price - trough_price) / ath_price * 100.0
    if drawdown_pct >= 5.0:
        duration_days = (trough_date - start_ath).days
        corrections.append({
            "ath_date":     start_ath,
            "trough_date":  trough_date,
            "drawdown_pct": drawdown_pct,
            "duration":     duration_days
        })

corr_df = pd.DataFrame(corrections).sort_values("ath_date").reset_index(drop=True)

if corr_df.empty:
    print("No corrections ≥ 5% found.")
else:
    durations = corr_df["duration"]
    p25, p50, p75 = np.percentile(durations, [25, 50, 75])
    print(f"25th pct: {int(p25)} days, median: {int(p50)} days, 75th pct: {int(p75)} days")

25th pct: 21 days, median: 39 days, 75th pct: 89 days


## Question 4. [Stocks] Earnings Surprise Analysis for Amazon (AMZN)
Calculate the median 2-day percentage change in stock prices following positive earnings surprises days.

In [77]:
earn_df = pd.read_csv("ha1_Amazon.csv", delimiter=";")
# Convert Earnings Date to pandas Timestamp
earn_df["Earnings Date Original"] = earn_df["Earnings Date"]
# Define a parsing function that extracts “Month DD, YYYY” and then uses pd.to_datetime
def parse_earnings_date(s: str):
    """
    Given a string like “April 29, 2026 at 6 AM EDT” (or any variation),
    extract the “Month DD, YYYY” portion and parse it as a Timestamp.
    Returns pd.Timestamp or pd.NaT if it can’t parse.
    """
    if not isinstance(s, str):
        return pd.NaT
    
    # 3A) Use regex to capture the “Month DD, YYYY” at the start of the string.
    #     Pattern: ^\s*([A-Za-z]+ \d{1,2}, \d{4})
    m = re.match(r"^\s*([A-Za-z]+ \d{1,2}, \s*\d{4})", s)
    if not m:
        return pd.NaT
    
    date_part = m.group(1).strip()
    # 3B) Try parsing that date_part. If it fails, return NaT.
    try:
        return pd.to_datetime(date_part, format="%B %d, %Y", errors="raise")
    except (ValueError, TypeError):
        # Might fail if day/month written differently; try generic parser
        try:
            return pd.to_datetime(date_part, errors="coerce")
        except:
            return pd.NaT

# 4) Apply the parsing function row‐by‐row
earn_df["Earnings Date"] = earn_df["Earnings Date"].apply(parse_earnings_date)

earn_df

,Symbol,Company,Earnings Date,EPS Estimate,Reported EPS,Surprise (%),Earnings Date Original
0,AMZN,Amazon.com Inc,2026-04-29,-,-,-,"April 29, 2026 at 6 AM EDT"
1,AMZN,Amazon.com Inc,2026-02-04,-,-,-,"February 4, 2026 at 4 PM EST"
2,AMZN,Amazon.com Inc,2025-10-29,-,-,-,"October 29, 2025 at 6 AM EDT"
3,AMZN,Amazon.com Inc,2025-07-30,-,-,-,"July 30, 2025 at 4 PM EDT"
4,AMZN,"Amazon.com, Inc.",2025-05-01,???.36,???.59,+16.74,"May 1, 2025 at 4 PM EDT"
...,...,...,...,...,...,...,...
112,AMZN,"Amazon.com, Inc.",1998-04-27,-,-,+13.92,"April 27, 1998 at 12 AM EDT"
113,AMZN,"Amazon.com, Inc.",1998-01-22,-,-,+11.41,"January 22, 1998 at 12 AM EST"
114,AMZN,"Amazon.com, Inc.",1997-10-27,-,-,+13.29,"October 27, 1997 at 12 AM EST"
115,AMZN,"Amazon.com, Inc.",1997-07-10,-,-,+13.33,"July 10, 1997 at 12 AM EDT"


In [78]:
if "Surprise(%)" in earn_df.columns:
    # Remove any stray % signs and convert to float
    earn_df["Surprise(%)"] = earn_df["Surprise(%)"].astype(str).str.replace("%", "")
    earn_df["Surprise(%)"] = pd.to_numeric(earn_df["Surprise(%)"], errors="coerce")


# Filter for rows where Reported EPS > EPS Estimate (i.e. positive surprise).
earn_df["EPS Estimate"] = pd.to_numeric(earn_df["EPS Estimate"], errors="coerce")
earn_df["Reported EPS"]   = pd.to_numeric(earn_df["Reported EPS"],   errors="coerce")

earn_df["Positive"] = earn_df["Reported EPS"] > earn_df["EPS Estimate"]

positive_df = earn_df[earn_df["Positive"]].copy()
print(f"\nFound {len(positive_df)} positive‐surprise rows.\n")
positive_df.head()


Found 33 positive‐surprise rows.



,Symbol,Company,Earnings Date,EPS Estimate,Reported EPS,Surprise (%),Earnings Date Original,Positive
8,AMZN,"Amazon.com, Inc.",2024-04-30,0.83,0.98,+17.91,"April 30, 2024 at 4 PM EDT",True
9,AMZN,"Amazon.com, Inc.",2024-02-01,0.80,1.00,+24.55,"February 1, 2024 at 4 PM EST",True
10,AMZN,"Amazon.com, Inc.",2023-10-26,0.58,0.94,+60.85,"October 26, 2023 at 4 PM EDT",True
11,AMZN,"Amazon.com, Inc.",2023-08-03,0.35,0.65,+85.73,"August 3, 2023 at 4 PM EDT",True
12,AMZN,"Amazon.com, Inc.",2023-04-27,0.21,0.31,+46.36,"April 27, 2023 at 4 PM EDT",True


In [81]:
# Handling AMZN Stock prices
start_date = pd.to_datetime(earn_df["Earnings Date"].min()) - pd.Timedelta(days=180)
end_date   = "2025-05-01"

# Download via yfinance
price_df = yf.download("AMZN", start=start_date.strftime("%Y-%m-%d"), end=end_date)[["Close"]]

price_df


[*********************100%***********************]  1 of 1 completed


Price,Close
Ticker,AMZN
Date,
1997-05-15,0.097917
1997-05-16,0.086458
1997-05-19,0.085417
1997-05-20,0.081771
1997-05-21,0.071354
...,...
2025-04-24,186.539993
2025-04-25,188.990005


In [80]:

# Compute 2-day returns for every possible 3-day window
price_df["Close_t_minus_1"] = price_df["Close"].shift(1)
price_df["Close_t_plus_1"]  = price_df["Close"].shift(-1)

# But the above would label returns at wrong rows. Instead, do this:
price_df["Close_t"]   = price_df["Close"]
price_df["Close_t+2"] = price_df["Close"].shift(-2)

# Compute 2-day return = (Close_{t+2} / Close_{t}) – 1
price_df["Return_2d"] = price_df["Close_t+2"] / price_df["Close_t"] - 1
price_df

Price,Close,Close_t_minus_1,Close_t_plus_1,Close_t,Close_t+2,Return_2d
Ticker,AMZN,,,,,
Date,,,,,,
1997-05-15,0.097917,NaN,0.086458,0.097917,0.085417,-0.127659
1997-05-16,0.086458,0.097917,0.085417,0.086458,0.081771,-0.054211
1997-05-19,0.085417,0.086458,0.081771,0.085417,0.071354,-0.164639
1997-05-20,0.081771,0.085417,0.071354,0.081771,0.069792,-0.146494
1997-05-21,0.071354,0.081771,0.069792,0.071354,0.075000,0.051097
...,...,...,...,...,...,...
2025-04-24,186.539993,180.600006,188.990005,186.539993,187.699997,0.006219
2025-04-25,188.990005,186.539993,187.699997,188.990005,187.389999,-0.008466
